In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

In [3]:
X_all = np.load('../hispanic_colrect/X.npy')
Y1 = np.load('../hispanic_colrect/Y1.npy')
Y2 = np.load('../hispanic_colrect/Y2.npy')
Y5 = np.load('../hispanic_colrect/Y5.npy')

In [4]:
# shuffle
np.random.seed(97)
idx = np.random.permutation(len(X_all))
X = X_all[idx]
Y = Y5[idx]

TEST_SET_SIZE = 6000

X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)

In [5]:
# Features Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
std_scaler = StandardScaler()
X_train[:,-4:] = std_scaler.fit_transform(X_train[:,-4:])
X_test[:,-4:] = std_scaler.transform(X_test[:,-4:])

/Users/sam/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

def results(classifier,threshold=0.5):
    Y_pred_test = classifier.predict(X_test)
    Y_pred_test = (Y_pred_test>=threshold).astype(int)
    #Y_pred_test = np.rint(Y_pred_test)
    print("Test accuracy score: " + str(accuracy_score(Y_test.astype(int), Y_pred_test)))
    print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
    matrix = confusion_matrix(Y_test.astype(int), Y_pred_test)
    tn, fp, fn, tp = matrix.ravel()
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    g_mean = np.sqrt(sensitivity*specificity)
    print("PPV: " + str(ppv))
    print("NPV: " + str(npv))
    print("Sensitivity: " + str(sensitivity))
    print("Specificity: " + str(specificity))
    print("G-Mean: " + str(g_mean))
    print("Confusion matrix:\n" + str(matrix))

In [125]:
X_train[0,-4:]

array([0.05973328722727474, -0.12173440434140541, -0.46331309102856094,
       -0.12734246089730297], dtype=object)

In [151]:
from tensorflow import keras
from keras.layers import Input, Dense, Dropout
from keras.models import Sequential
from keras import backend as K

model = Sequential()
#model.add(Dropout(0.25, input_shape=(X.shape[1],)))
model.add(Dense(400, input_dim=X.shape[1], activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))
model.add(Dense(400, activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))
model.add(Dense(400, activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))
model.add(Dense(1, activation='tanh'))

In [171]:
#with tf.Session(graph=graph) as sess:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=4, batch_size=50)
#scores = model.evaluate(X_test, Y_test)
#print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/4
40785/40785 [==============================] - 13s 329us/step - loss: 0.3946 - acc: 0.8302
Epoch 2/4
40785/40785 [==============================] - 12s 286us/step - loss: 0.3908 - acc: 0.8293
Epoch 3/4
40785/40785 [==============================] - 12s 305us/step - loss: 0.3795 - acc: 0.8335
Epoch 4/4
6000/6000 [==============================] - 2s 310us/step

acc: 81.82%


In [176]:
results(model,threshold=0.5)

Test accuracy score: 0.820333333333
ROC: 0.73051095839
PPV: 0.866456361725
NPV: 0.644176706827
Sensitivity: 0.902914749069
Specificity: 0.558107167711
G-Mean: 0.70987547731
Confusion matrix:
[[ 802  635]
 [ 443 4120]]


In [133]:
Y_pred_test = model.predict(X_test)

In [134]:
Y_pred_test[:50]

array([[ 784.01721191],
       [ 528.52581787],
       [ 187.64071655],
       [ 733.89245605],
       [ 647.36193848],
       [ 683.46484375],
       [ 583.29669189],
       [ 693.15026855],
       [ 940.70153809],
       [ 493.43051147],
       [ 355.34234619],
       [  34.22605515],
       [ 557.21331787],
       [ 562.82177734],
       [ 352.20883179],
       [ 646.42791748],
       [ 431.49417114],
       [ -16.5065403 ],
       [ 380.70178223],
       [ 632.74365234],
       [ 483.49789429],
       [  -3.92847538],
       [ 704.1003418 ],
       [ 619.94293213],
       [ 515.65936279],
       [ 567.6628418 ],
       [  -8.33492661],
       [ 255.03117371],
       [-136.08525085],
       [ 558.50848389],
       [ 145.27659607],
       [ 578.0199585 ],
       [ 594.91137695],
       [ 750.55444336],
       [ 640.38830566],
       [ 713.97802734],
       [ 573.45690918],
       [ 604.72644043],
       [ 524.90649414],
       [ 458.16394043],
       [ 532.83996582],
       [ 483.971

In [12]:
#with graph.as_default():
Y_pred_test = model.predict(X_test)
Y_pred_test = np.rint(Y_pred_test)

In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
#Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

0.823833333333
ROC: 0.738294587901


array([[ 825,  612],
       [ 445, 4118]])

In [130]:
tn, fp, fn, tp = confusion_matrix(Y_test.astype(int), Y_pred_test).ravel()
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print("PPV: " + str(ppv))
print("NPV: " + str(npv))
print("Sensitivity: " + str(sensitivity))
print("Specificity: " + str(specificity))

PPV: 0.873648505406
NPV: 0.655494933749
Sensitivity: 0.903133903134
Specificity: 0.58524704245
